Final Project Submission <br>
Student name: Qilun Chen, Evan Serrano<br>
Student pace: full time<br>
Scheduled project review date/time: April/1/2022<br>
Instructor name: Praveen Gowtham, Joe Comeaux<br>
Blog post URL:https://github.com/nkbuddy/dsc-phase-3-project-NBA

### Table of Contents
* [STEP 1: Define the Problem](#Define-the-Problem)
* [Step 2: Gather the Data](#Gather-the-Data)
* [Step 3: Prepare Data for Consumption](#Prepare-Data-for-Consumption)
    * [3.1 Import Libraries](#Import-Libraries)
        * [3.11 Load Data Modelling Libraries](#Load-Data-Modelling-Libraries)
    * [3.2 Meet and Greet Data](#Meet-and-Greet-Data)
        * [3.21 The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting](#4C)
        * [3.22 Clean Data](#Clean-Data)
        * [3.23 Convert Formats](#Convert-Formats)
        * [3.24 Da-Double Check Cleaned Data](#Da-Double-Check-Cleaned-Data)
        * [3.25 Split Training and Testing Data](#Split-Training-and-Testing-Data)
* [Step 4: Perform Exploratory Analysis with Statistics](#Perform-Exploratory-Analysis-with-Statistics)
* [Step 5: Model Data](#Model-Data)
    * [5.1 Evaluate Model Performance](#Evaluate-Model-Performance)
        * [5.11 Model Performance with Cross-Validation (CV)](#CV)
        * [5.12 Tune Model with Hyper-Parameters](#Tune-Model-with-Hyper-Parameters)
        * [5.13 Tune Model with Feature Selection](#Tune-Model-with-Feature-Selection)
* [Step 6: Validate and Implement](#Validate-and-Implement)
* [STEP 7: Optimize and Strategize](#Optimize-and-Strategize)

# STEP 1: Define the Problem <a class="anchor" id="Define-the-Problem"></a>

Giving a recommendation to people who has favortie coffee but wants to try sometings news.

# Step 2: Gather the Data <a class="anchor" id="Gather-the-Data"></a>

Web scraping from coffeereview.com

# Step 3: Prepare Data for Consumption <a class="anchor" id="Prepare-Data-for-Consumption"></a>

## 3.1 Import Libraries <a class="anchor" id="Import-Libraries"></a>

In [11]:
import numpy as np
import seaborn as sns
import numpy as np
from copy import deepcopy

#visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# NLP modules we will use for text normalization
import re #regex 
import nltk # the natural language toolkit
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag

# feature construction
from sklearn.feature_extraction.text import TfidfVectorizer #use this to create BoW matrix

from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from sklearn.manifold import TSNE
import nltk
nltk.download('punkt')
np.random.seed(0)

[nltk_data] Downloading package punkt to /Users/alanchan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3.11 Load Data Modelling Libraries <a class="anchor" id="Load-Data-Modelling-Libraries"></a>

In [12]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')

In [13]:
#modeling and dimensionality reduction for visuaization
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE

## 3.2 Meet and Greet Data <a class="anchor" id="Meet-and-Greet-Data"></a>

1) coffee_score is number. The range is 0-100. higher is better.
2) coffee_roaster_name is string. factory name.
3) coffee_bean_name is string. coffe bean name.
4) roaster_location is string. factory location.
5) coffee_origin is string. coffee beans from which country.
6) roast_level is string. different level of roast.
7) agtron is number. number of roast level.
8) estimate_price is number. the units are USD/OZ
9) review_date is day. The day wrote the review.
10) aroma is number. The range is 0-10. higher number mean the smell of coffee beans is better.
11) acidity_or_structure is number. The range is 0-10. higher number mean the coffee test is sour.
12) body is number. The range is 0-10. higher number mean the mouthfeel is better.
13) flavor is number. The range is 0-10. higher number mean strong flavor.
14) aftertaste is number. The range is 0-10. higher number mean better aftertaste.
15) text is string. All comments about the coffee.

In [276]:
coffee_df = pd.read_csv("coffee_review")

In [277]:
print(coffee_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5748 entries, 0 to 5747
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            5748 non-null   int64  
 1   coffee_score          5748 non-null   object 
 2   coffee_roaster_name   5748 non-null   object 
 3   coffee_bean_name      5748 non-null   object 
 4   roaster_location      5746 non-null   object 
 5   coffee_origin         5236 non-null   object 
 6   roast_level           5114 non-null   object 
 7   agtron                5236 non-null   object 
 8   estimate_price        3734 non-null   object 
 9   review_date           5681 non-null   object 
 10  aroma                 5646 non-null   object 
 11  acidity_or_structure  1382 non-null   float64
 12  body                  4919 non-null   object 
 13  flavor                4917 non-null   object 
 14  aftertaste            4280 non-null   float64
 15  blind_assessment     

In [278]:
coffee_df.head()

,Unnamed: 0,coffee_score,coffee_roaster_name,coffee_bean_name,roaster_location,coffee_origin,roast_level,agtron,estimate_price,review_date,aroma,acidity_or_structure,body,flavor,aftertaste,blind_assessment,notes,bottom_line
0,0,94,Nostalgia Coffee Roasters,Rwanda Gatare,"San Diego, California","Nyamasheke District, Western Province, Rwanda",Medium-Light,58/80,$22.00/12 ounces,May 2022,9,9.0,9\t\t\t\t\t\t\t,9,8.0,"Blind Assessment: Very chocolaty, rich-toned. ...",Notes: Produced by smallholding farmers surrou...,"The Bottom Line: A confident, complex Rwanda c..."
1,1,93,Yellow Brick Coffee,Guatemala ASPROCDEGUA Women Producers,"Tucson, Arizona","Huehuetenango, Guatemala",Medium-Light,58/77,$18.00/12 ounces,May 2022,9,8.0,9\t\t\t\t\t\t\t,9,8.0,"Blind Assessment: Crisply chocolaty, nut-toned...",Notes: Produced by 664 smallholding women farm...,"The Bottom Line: A balanced, inviting women-fa..."
2,2,94,Coffee by Design,Costa Rica Naranjo Danilo Salazar Arias,"Portland, Maine","Alajuela, Costa Rica",Medium-Light,60/77,$23.00/12 ounces,May 2022,9,9.0,9\t\t\t\t\t\t\t,9,8.0,"Blind Assessment: Bright, richly sweet-tart, f...",Notes: Produced by Danilo Salazar Arias of San...,"The Bottom Line: A lively, juicy-bright natura..."
3,3,93,Coffee by Design,Kenya Kahunyo AA,"Portland, Maine","Nyeri growing region, south-central Kenya",Medium-Light,59/76,$22.00/16 ounces,May 2022,9,9.0,8\t\t\t\t\t\t\t,9,8.0,"Blind Assessment: Crisply sweet-savory. Date, ...","Notes: Despite grower discontent, unclear gove...","The Bottom Line: A savory, richly Kenya cup w..."
4,4,93,Coffee by Design,Honduras COMSA Oscar Omar Alonzo,"Portland, Maine","Marcala, Honduras",Medium-Light,58/74,$18.50/12 ounces,May 2022,9,8.0,9\t\t\t\t\t\t\t,9,8.0,Blind Assessment: A balanced honey-processed c...,Notes: Produced by Oscar Omar Alonzo of Finca ...,"The Bottom Line: Delicately sweet-tart, deeply..."


### 3.21 The 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting <a class="anchor" id="4C"></a>

In [279]:
print('columns with null values:\n', coffee_df.isnull().sum())
print("-"*10)
coffee_df.describe(include='all')

columns with null values:
 Unnamed: 0                 0
coffee_score               0
coffee_roaster_name        0
coffee_bean_name           0
roaster_location           2
coffee_origin            512
roast_level              634
agtron                   512
estimate_price          2014
review_date               67
aroma                    102
acidity_or_structure    4366
body                     829
flavor                   831
aftertaste              1468
blind_assessment           0
notes                      0
bottom_line             3837
dtype: int64
----------


,Unnamed: 0,coffee_score,coffee_roaster_name,coffee_bean_name,roaster_location,coffee_origin,roast_level,agtron,estimate_price,review_date,aroma,acidity_or_structure,body,flavor,aftertaste,blind_assessment,notes,bottom_line
count,5748.000000,5748,5748,5748,5746,5236,5114,5236,3734,5681,5646,1382.000000,4919,4917,4280.000000,5748,5748,1911
unique,NaN,38,1128,4742,735,1786,6,1031,1068,288,35,NaN,32,40,NaN,5743,5539,1910
top,NaN,93,JBC Coffee Roasters,Ethiopia Yirgacheffe,"Madison, Wisconsin",Not disclosed.,Medium-Light,58/76,$18.00/12 ounces,August 2020,9,NaN,8\t\t\t\t\t\t\t,9,NaN,Blind Assessment: Reader Peter Lynagh nominate...,Notes: Tully's is a large quality-oriented spe...,The Bottom Line: A cocoa-toned natural-process...
freq,NaN,1002,302,26,326,347,2248,88,147,50,2622,NaN,2417,2699,NaN,2,7,2
mean,2873.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.429812,NaN,NaN,7.926869,NaN,NaN,NaN
std,1659.449005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.662807,NaN,NaN,0.764073,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,2.000000,NaN,NaN,NaN
25%,1436.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,8.000000,NaN,NaN,NaN
50%,2873.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.000000,NaN,NaN,8.000000,NaN,NaN,NaN
75%,4310.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,NaN,8.000000,NaN,NaN,NaN


In [280]:
coffee_df['acidity_or_structure'].unique()

array([ 9.,  8.,  7.,  6., nan, 10.,  3.,  1.,  2.])

### 3.22 Clean Data <a class="anchor" id="Clean-Data"></a>

In [342]:
cleaned_df = coffee_df.dropna()

In [343]:
cleaned_df.loc[:, 'coffee_score'] = cleaned_df["coffee_score"].astype('int64')
cleaned_df.loc[:, 'coffee_roaster_name'] = cleaned_df["coffee_roaster_name"].astype('string')
cleaned_df.loc[:, 'coffee_bean_name'] = cleaned_df["coffee_bean_name"].astype('string')
cleaned_df.loc[:, 'coffee_roaster_name'] = cleaned_df["coffee_roaster_name"].astype('string')
cleaned_df.loc[:, 'roaster_location'] = cleaned_df["roaster_location"].astype('string')
cleaned_df.loc[:, 'coffee_origin'] = cleaned_df["coffee_origin"].astype('string')
cleaned_df.loc[:, 'roast_level'] = cleaned_df["roast_level"].astype('string')
cleaned_df.loc[:, 'agtron'] = cleaned_df["agtron"].astype('string')
cleaned_df.loc[:, 'estimate_price'] = cleaned_df["estimate_price"].astype('string')
cleaned_df.loc[:, 'review_date'] = cleaned_df["review_date"].astype('string')
cleaned_df.loc[:, 'aroma'] = cleaned_df["aroma"].astype('int64')
cleaned_df.loc[:, 'acidity_or_structure'] = cleaned_df["acidity_or_structure"].astype('int64')
cleaned_df.loc[:, 'body'] = cleaned_df["body"].str.replace("\t\t\t\t\t\t\t", "").astype('int64')
cleaned_df.loc[:, 'flavor'] = cleaned_df["flavor"].astype('int64')
cleaned_df.loc[:, 'aftertaste'] = cleaned_df["aftertaste"].astype('int64')
cleaned_df.loc[:, 'blind_assessment'] = cleaned_df["blind_assessment"].astype('string')
cleaned_df.loc[:, 'notes'] = cleaned_df["notes"].astype('string')
cleaned_df.loc[:, 'bottom_line'] = cleaned_df["bottom_line"].astype('string')

/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [344]:
cleaned_df.drop(cleaned_df[cleaned_df['roast_level'] == 'NA'].index, inplace = True)
cleaned_df.drop(cleaned_df[cleaned_df['agtron'] == 'NA/NA'].index, inplace = True)

/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [345]:
drop_column = ['Unnamed: 0','roaster_location','review_date','notes']
cleaned_df.drop(drop_column, axis=1, inplace = True)

In [346]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1362 entries, 0 to 1549
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   coffee_score          1362 non-null   int64 
 1   coffee_roaster_name   1362 non-null   string
 2   coffee_bean_name      1362 non-null   string
 3   coffee_origin         1362 non-null   string
 4   roast_level           1362 non-null   string
 5   agtron                1362 non-null   string
 6   estimate_price        1362 non-null   string
 7   aroma                 1362 non-null   int64 
 8   acidity_or_structure  1362 non-null   int64 
 9   body                  1362 non-null   int64 
 10  flavor                1362 non-null   int64 
 11  aftertaste            1362 non-null   int64 
 12  blind_assessment      1362 non-null   string
 13  bottom_line           1362 non-null   string
dtypes: int64(6), string(8)
memory usage: 159.6 KB


In [347]:
cleaned_df['min_agtron'] = cleaned_df['agtron'].map(lambda x: x.split('/')[0])
cleaned_df['max_agtron'] = cleaned_df['agtron'].map(lambda x: x.split('/')[1])

<ipython-input-347-603df9194159>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['min_agtron'] = cleaned_df['agtron'].map(lambda x: x.split('/')[0])
<ipython-input-347-603df9194159>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['max_agtron'] = cleaned_df['agtron'].map(lambda x: x.split('/')[1])


In [348]:
cleaned_df.drop(cleaned_df[cleaned_df['estimate_price'].str.contains('ounces') == False].index, inplace = True)
cleaned_df.drop(cleaned_df[cleaned_df['estimate_price'].str.contains('NT') == True].index, inplace = True)
cleaned_df.drop(cleaned_df[cleaned_df['estimate_price'].str.contains('CAD') == True].index, inplace = True)

/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [349]:
cleaned_df['estimate_price_USD'] = cleaned_df['estimate_price'].map(lambda x: x.split('.')[0])
cleaned_df['estimate_price_ounces'] = cleaned_df['estimate_price'].map(lambda x: x.split('/')[1])

<ipython-input-349-7740106d8e1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['estimate_price_USD'] = cleaned_df['estimate_price'].map(lambda x: x.split('.')[0])
<ipython-input-349-7740106d8e1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['estimate_price_ounces'] = cleaned_df['estimate_price'].map(lambda x: x.split('/')[1])


In [350]:
cleaned_df['estimate_price_USD'].unique()

array(['$22', '$18', '$23', '$35', '$16', '$28', '$24', '$25', '$19',
       '$20', '$17', '$30', '$26', '$140', '$21', '$65', '$14', '$13',
       '$15', '$12', '$9', '$48', '$69', '$40', '$42', '$36', '$49',
       '$11', '$45', '$34', '$38', '$29', '$50', '$10', '$5', '$75',
       '$89', '$150', '$145', '$62', '$95', '$60', '$120', '$32', '$27',
       '$80', '$54', '$39', '$74', '$87', '$33', '$47', '$56', '$55',
       '$85', '$20/12 ounces', '$64', 'US $20', '$6', '$4', '$8', '$125',
       '18', '$31', '$63', '$37', '$44', '#23', '$70', '$51'],
      dtype=object)

In [351]:
cleaned_df.loc[:, 'estimate_price_USD'] = cleaned_df["estimate_price_USD"].str.replace("$", "")
cleaned_df.loc[:, 'estimate_price_USD'] = cleaned_df["estimate_price_USD"].str.replace("/12 ounces", "")
cleaned_df.loc[:, 'estimate_price_USD'] = cleaned_df["estimate_price_USD"].str.replace("US ", "")
cleaned_df.loc[:, 'estimate_price_USD'] = cleaned_df["estimate_price_USD"].str.replace("#", "")

/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [356]:
cleaned_df['estimate_price_ounces'].unique()

array([12.  , 16.  ,  8.  ,  5.  ,  4.  ,  7.  , 24.  ,  6.  , 10.5 ,
       10.  , 12.64, 11.  , 14.  ,  2.  , 20.  , 22.  , 32.  ])

In [353]:
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace(" ", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("\(", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("\)", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("\"", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("$", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace(";", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("ounces", "")

In [354]:
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace(" ", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("\(", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("\)", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("\"", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("$", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace(";", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("ounces", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("2types", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("currentlyonsalefor36.76", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("35.00", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("79.00", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("includesshipping", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("50.00", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("limitedavailability", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("packagedasaduowithBourbonReyGuatemala", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("packagedasaduowiththeBourbonReyJamaica", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("*", "")
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].str.replace("online12.99", "")

In [355]:
cleaned_df.loc[:, 'estimate_price_USD'] = cleaned_df["estimate_price_USD"].astype('float')
cleaned_df.loc[:, 'estimate_price_ounces'] = cleaned_df["estimate_price_ounces"].astype('float64')

In [364]:
cleaned_df['estimate_price_USD_per_ounces'] = round(cleaned_df["estimate_price_USD"] / cleaned_df["estimate_price_ounces"], 2)

<ipython-input-364-f04327590dec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['estimate_price_USD_per_ounces'] = round(cleaned_df["estimate_price_USD"] / cleaned_df["estimate_price_ounces"], 2)


In [367]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1278 entries, 0 to 1549
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   coffee_score                   1278 non-null   int64  
 1   coffee_roaster_name            1278 non-null   string 
 2   coffee_bean_name               1278 non-null   string 
 3   coffee_origin                  1278 non-null   string 
 4   roast_level                    1278 non-null   string 
 5   agtron                         1278 non-null   string 
 6   estimate_price                 1278 non-null   string 
 7   aroma                          1278 non-null   int64  
 8   acidity_or_structure           1278 non-null   int64  
 9   body                           1278 non-null   int64  
 10  flavor                         1278 non-null   int64  
 11  aftertaste                     1278 non-null   int64  
 12  blind_assessment               1278 non-null   s

In [368]:
drop_column = ['agtron','estimate_price','estimate_price_USD','estimate_price_ounces']
cleaned_df.drop(drop_column, axis=1, inplace = True)

/Users/alanchan/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### 3.23 Convert Formats <a class="anchor" id="Convert-Formats"></a>

In [381]:
cleaned_df.describe(include='all')

,coffee_score,coffee_roaster_name,coffee_bean_name,coffee_origin,roast_level,aroma,acidity_or_structure,body,flavor,aftertaste,blind_assessment,bottom_line,min_agtron,max_agtron,estimate_price_USD_per_ounces
count,1278.000000,1278,1278,1278,1278,1278.000000,1278.000000,1278.000000,1278.000000,1278.000000,1278,1278,1278,1278,1278.000000
unique,NaN,247,1161,624,5,NaN,NaN,NaN,NaN,NaN,1277,1277,40,51,NaN
top,NaN,JBC Coffee Roasters,Ethiopia Kayon Mountain,"Yirgacheffe growing region, southern Ethiopia",Medium-Light,NaN,NaN,NaN,NaN,NaN,"Blind Assessment: Richly chocolaty, deeply swe...",The Bottom Line: A cocoa-toned natural-process...,58,78,NaN
freq,NaN,130,5,48,941,NaN,NaN,NaN,NaN,NaN,2,2,247,196,NaN
mean,92.899844,NaN,NaN,NaN,NaN,8.812989,8.453834,8.602504,8.953052,8.075900,NaN,NaN,NaN,NaN,2.533513
std,1.727111,NaN,NaN,NaN,NaN,0.440958,0.574024,0.516048,0.361827,0.494003,NaN,NaN,NaN,NaN,2.924548
min,84.000000,NaN,NaN,NaN,NaN,7.000000,6.000000,7.000000,7.000000,6.000000,NaN,NaN,NaN,NaN,0.250000
25%,92.000000,NaN,NaN,NaN,NaN,9.000000,8.000000,8.000000,9.000000,8.000000,NaN,NaN,NaN,NaN,1.330000
50%,93.000000,NaN,NaN,NaN,NaN,9.000000,8.000000,9.000000,9.000000,8.000000,NaN,NaN,NaN,NaN,1.580000
75%,94.000000,NaN,NaN,NaN,NaN,9.000000,9.000000,9.000000,9.000000,8.000000,NaN,NaN,NaN,NaN,2.250000


In [398]:
cleaned_df['best_value'] = round(cleaned_df['coffee_score'] / cleaned_df['estimate_price_USD_per_ounces'], 2)

<ipython-input-398-6e7481e717f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['best_value'] = round(cleaned_df['coffee_score'] / cleaned_df['estimate_price_USD_per_ounces'], 2)


In [399]:
cleaned_df.sort_values(by='best_value',ascending=False).head(10)

,coffee_score,coffee_roaster_name,coffee_bean_name,coffee_origin,roast_level,aroma,acidity_or_structure,body,flavor,aftertaste,blind_assessment,bottom_line,min_agtron,max_agtron,estimate_price_USD_per_ounces,best_value
889,87,Hill's Bros. Coffee,100% Guatemalan,Guatemala,Medium-Dark,8,7,7,8,7,"Blind Assessment: Sweetly nut-toned, roasty. H...",The Bottom Line: A sweet though muted Guatemal...,41,53,0.25,348.00
887,89,HT Traders,Colombia,Colombia,Medium-Dark,8,7,8,9,7,"Blind Assessment: Rich-toned, chocolaty. Dark ...","The Bottom Line: A comfortingly familiar, dark...",40,56,0.33,269.70
886,89,Cameron’s Coffee,Guatemala (from bulk bin),Guatemala,Medium,8,7,8,9,7,"Blind Assessment: Balanced, sweet-toned. Bakin...",The Bottom Line: A good-value darker roasted G...,44,60,0.38,234.21
888,88,Eight O'Clock Coffee,Colombia,Colombia,Medium,8,7,8,8,7,"Blind Assessment: Crisply sweet, cocoa-toned. ...","The Bottom Line: A roast-prominent, cocoa-tone...",46,60,0.41,214.63
163,89,El Gran Cafe,5a Sur,"Antigua, Guatemala",Medium,8,7,8,9,7,"Blind Assessment: Crisply sweet, nut-toned. Ca...",The Bottom Line: A darkish medium version of a...,52,64,0.42,211.90
159,88,El Gran Cafe,5a Poniente,"Antigua, Guatemala",Medium,8,7,8,8,7,Blind Assessment: Richly bittersweet. Dark cho...,"The Bottom Line: A sweet, chocolaty, earthy Gu...",48,56,0.42,209.52
611,86,El Gran Café,5a Poniente,"Antigua growing region, Guatemala",Medium-Dark,7,7,7,8,7,"Blind Assessment: Chocolaty, sweetly smoky. Da...",The Bottom Line: A classic dark roast Guatemal...,42,56,0.42,204.76
610,86,El Gran Café,5a Sur,"Antigua growing region, Guatemala",Medium,7,7,7,8,7,Blind Assessment: Sweetly nut- and cocoa-toned...,"The Bottom Line: An accessible, sweetly nutty ...",50,68,0.42,204.76
758,93,Columbia Street Roastery,40-Hour Ethiopia Girma Eshetu,"Bita Genet District, Keffa Zone, southwestern ...",Medium-Light,9,8,9,9,8,"Blind Assessment: Richly sweet-savory, complex...",The Bottom Line: A bargain opportunity for cof...,56,72,0.75,124.00
56,87,Peet's Coffee,Major Dickason’s Blend,Americas; Indo-Pacific,Medium-Dark,8,7,8,8,6,"Blind Assessment: Rather dry, wood-framed, bit...","The Bottom Line: The popular Peet’s blend, her...",38,50,0.75,116.00


In [427]:
cleaned_df[coffee_df['notes'].str.contains('Arabica')]

<ipython-input-427-7718538084d8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_df[coffee_df['notes'].str.contains('Arabica')]


,coffee_score,coffee_roaster_name,coffee_bean_name,coffee_origin,roast_level,aroma,acidity_or_structure,body,flavor,aftertaste,blind_assessment,bottom_line,min_agtron,max_agtron,estimate_price_USD_per_ounces,best_value
0,94,Nostalgia Coffee Roasters,Rwanda Gatare,"Nyamasheke District, Western Province, Rwanda",Medium-Light,9,9,9,9,8,"Blind Assessment: Very chocolaty, rich-toned. ...","The Bottom Line: A confident, complex Rwanda c...",58,80,1.83,51.37
1,93,Yellow Brick Coffee,Guatemala ASPROCDEGUA Women Producers,"Huehuetenango, Guatemala",Medium-Light,9,8,9,9,8,"Blind Assessment: Crisply chocolaty, nut-toned...","The Bottom Line: A balanced, inviting women-fa...",58,77,1.50,62.00
2,94,Coffee by Design,Costa Rica Naranjo Danilo Salazar Arias,"Alajuela, Costa Rica",Medium-Light,9,9,9,9,8,"Blind Assessment: Bright, richly sweet-tart, f...","The Bottom Line: A lively, juicy-bright natura...",60,77,1.92,48.96
4,93,Coffee by Design,Honduras COMSA Oscar Omar Alonzo,"Marcala, Honduras",Medium-Light,9,8,9,9,8,Blind Assessment: A balanced honey-processed c...,"The Bottom Line: Delicately sweet-tart, deeply...",58,74,1.50,62.00
5,92,Coffee by Design,Rwanda Gasharu Natural,"Nyamasheke District, Rwanda",Medium-Light,8,8,9,9,8,"Blind Assessment: Delicately fruit-forward, cr...","The Bottom Line: A friendly, easygoing Rwanda ...",58,76,1.92,47.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,94,Lexington Coffee Roasters,Kapsakiso Kenya,"Bungoma region, western Kenya",Light,9,9,9,9,8,"Blind Assessment: Deeply rich, crisply sweet-s...","The Bottom Line: An enticing, classic Kenya: b...",60,84,1.50,62.67
1546,93,Caffe Luxxe,Ethiopia Oromia,"Shakiso growing district, Guji Zone, Oromia Re...",Medium,9,8,9,9,8,"Blind Assessment: Richly sweet, fruit-toned. D...",The Bottom Line: This natural-processed Ethiop...,49,72,1.67,55.69
1547,92,Paradise Roasters,Congo Natural,"Kalehe, South Kivu Province, Democratic Republ...",Medium-Light,8,8,9,9,8,Blind Assessment: Sweetly spice-toned. Grapefr...,"The Bottom Line: A crisp, spice-framed Congo n...",60,78,1.33,69.17
1548,92,Conscious Cup Coffee Roasters,Ethiopia Yirgacheffe,"Yirgacheffe growing region, southern Ethiopia",Medium-Light,9,8,8,9,8,"Blind Assessment: Delicate, richly sweet. Peac...",The Bottom Line: An attractive washed Yirgache...,53,76,1.38,66.67


#### Natural Language Processing

In [373]:
coffee_token = pd.DataFrame(cleaned_df, columns=["bottom_line"])

In [374]:
coffee_token['bottom_line'][0]

'The Bottom Line: A confident, complex Rwanda cup with rich fruit, deep chocolate, and spicy floral notes.'

In [376]:
coffee_token['bottom_line'] = coffee_token['bottom_line'].str.lower()

In [377]:
coffee_token.iloc[100]["bottom_line"]

'the bottom line: a familiar, comforting blend with leading notes of dark caramel, rich nut tones, and pretty florals — a classic daily drinker.'

In [378]:
coffee_flavor_wheel_first_layer=['floral', 'fruity', 'sour/fermented', 'green/vegetative', 'other', 'roasted', 'spices', 'nutty/cocoa', 'sweet']
coffee_flavor_wheel_second_layer=['black_tea', 'floral', 'berry', 'dried_fruit', 'other_fruit', 'citrus_fruit', 'sour', 'alcohol/fermented', 'olive oil', 'raw'\
    , 'green/vegetative', 'beany', 'papery/musty', 'chemical', 'pipe_tobacco', 'tobacco', 'burnt', 'cereal', 'pungent', 'pepper',\
         'brown_spice', 'nutty', 'cocoa', 'brown_sugar', 'vanilla', 'vanillin', 'overall_sweet', 'sweet_aromatics']
coffee_flavor_wheel_third_layer=['chamomile', 'rose', 'jasmine', 'blackberry', 'raspberry', 'blueberry', 'strawberry', 'raisin', 'prune', 'coconut', 'cherry',\
'pomegranate', 'pineapple', 'grape', 'apple', 'peach', 'pear', 'grapefruit', 'orange', 'lemon', 'lime', 'sour_aromatics', 'acetic_acid', 'butyric_acid', 'critric_acid',\
'malic_acid', 'winey', 'whisley', 'fermented', 'overripe', 'under-ripe', 'peapod', 'fresh', 'dark_green', 'vegetative', 'hay-like', 'herb-like', 'stable', 'cardboard',\
'papery', 'woody', 'moldy/damp', 'musty/dusty', 'musty/earthy', 'animalic', 'meaty/brothy', 'phenolic', 'bitter', 'salty', 'medicinal', 'petroleum', 'skunky', 'rubber',\
'acrid', 'ashy', 'smoky', 'brown', 'grain', 'malt', 'anise', 'nutmeg', 'cinnamon', 'clove', 'peanuts', 'hazelnut', 'almond', 'chocolate', 'dark_chocolate', 'molasses', 'mapple_syrup', 'caramelized', 'honey']

In [379]:
len(coffee_flavor_wheel_third_layer)

72

++++++++++

In [ ]:
label = LabelEncoder()
cleaned_df['roast_level_Code'] = label.fit_transform(cleaned_df['roast_level'])

Target = ['coffee_score']

### 3.24 Da-Double Check Cleaned Data <a class="anchor" id="Da-Double-Check-Cleaned-Data"></a>

In [473]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1290 entries, 0 to 1548
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   coffee_score          1290 non-null   int64 
 1   coffee_roaster_name   1290 non-null   object
 2   coffee_bean_name      1290 non-null   object
 3   coffee_origin         1290 non-null   object
 4   roast_level           1290 non-null   object
 5   agtron                1290 non-null   object
 6   estimate_price        1290 non-null   object
 7   aroma                 1290 non-null   int64 
 8   acidity_or_structure  1290 non-null   int64 
 9   body                  1290 non-null   int64 
 10  flavor                1290 non-null   int64 
 11  aftertaste            1290 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 131.0+ KB


### 3.25 Split Training and Testing Data <a class="anchor" id="Split-Training-and-Testing-Data"></a>

# Step 4: Perform Exploratory Analysis with Statistics <a class="anchor" id="Perform-Exploratory-Analysis-with-Statistics"></a>

# Step 5: Model Data <a class="anchor" id="Model-Data"></a>

## 5.1 Evaluate Model Performance <a class="anchor" id="Evaluate-Model-Performance"></a>

### 5.11 Model Performance with Cross-Validation (CV) <a class="anchor" id="CV"></a>

### 5.12 Tune Model with Hyper-Parameters <a class="anchor" id="Tune-Model-with-Hyper-Parameters"></a>

### 5.13 Tune Model with Feature Selection <a class="anchor" id="Tune-Model-with-Feature-Selection"></a>

# Step 6: Validate and Implement <a class="anchor" id="Validate-and-Implement"></a>

# Step 7: Optimize and Strategize<a class="anchor" id="Optimize-and-Strategize"></a>

### Conclusion
Iteration one of the Data Science Framework, seems to converge on 0.77990 submission accuracy. Using the same dataset and different implementation of a random forest (adaboost,decision tree, gradient boost, xgboost, etc.) with tuning does not exceed the 0.77990 submission accuracy. Interesting for this dataset, the simple random forest algorithm had the best default submission score and with tuning achieved the same best accuracy score.

While no general conclusions can be made from testing a handful of algorithms on a single dataset, there are several observations on the mentioned dataset.

The train dataset has a different distribution than the test/validation dataset and population. This created wide margins between the cross validation (CV) accuracy score .
Given the same dataset, random forest based algorithms, seemed to converge on the same accuracy score after proper tuning.
Despite tuning, no machine learning algorithm, exceeded the homemade algorithm. The author will theorize, that for small datasets, a manmade algorithm is the bar to beat.
With that in mind, for iteration two, I would spend more time on preprocessing and feature engineering. In order to better align the CV score and improve the overall accuracy.